<a href="https://colab.research.google.com/github/sdossou/Whats_his_name/blob/main/Whats_his_name_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "What's His Name" Application using LangGraph Tools

"What's His Name" application allows a user to look up a famous person's name based on their characteristics.

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

This notebook uses the Wikidata and the Duck Duck Go Web Search tools.



## Dependencies

Installing relevant dependencies and setting API keys for openAI and LangSmith.

In [ ]:
!pip install -qU langchain langchain_openai langgraph wikibase-rest-api-client mediawikiapi duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


In [ ]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"LangGraph Demo - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

LangSmith API Key: ··········


## Tool Selection

### Creating the Tool Belt

Installing the agent with a toolbelt to help answer questions and add external knowledge with wikidata and Duck Duck Go Web Search.


In [ ]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun

tool_belt = [DuckDuckGoSearchRun(), WikidataQueryRun(api_wrapper=WikidataAPIWrapper())]

### Actioning with Tools

Setting up the ToolExecutor to run the process.

In [ ]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tool_belt)

### Model

Setting-up the OpenAI model.

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)

Binding the LangChain formatted tools to the model in an OpenAI function calling format.

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tool_belt]
model = model.bind_functions(functions)

## Agent State

`coordinated multi-actor and stateful applications`

"stateful" means that we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion, we want to be able to ensure that we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

This `AgentState` object is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means:

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object

In [ ]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], operator.add]

## Graph Setup

Setting up the graph (nodes and edges) with the state, the tools and the LLM previously defined.


In [ ]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

async def call_model(state):
  messages = state["messages"]
  response = await model.ainvoke(messages)
  return {"messages" : [response]}

async def call_tool(state):
  last_message = state["messages"][-1]

  action = ToolInvocation(
      tool=last_message.additional_kwargs["function_call"]["name"],
      tool_input=json.loads(
          last_message.additional_kwargs["function_call"]["arguments"]
      )
  )

  response = await tool_executor.ainvoke(action)

  function_message = FunctionMessage(content=str(response), name=action.tool)

  return {"messages" : [function_message]}

Defining 2 nodes:

- `call_model` is a node that calls the model
- `call_tool` is a node which calls a tool



In [ ]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

Adding the entrypoint

In [ ]:
workflow.set_entry_point("agent")

Setting up a "conditional edge" which will use the output state of a node to determine which path to follow.

Creating an edge where the origin node is the agent node and the destination node can be either the action node or the END (finish the graph).

The dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [ ]:
def should_continue(state):
  last_message = state["messages"][-1]

  if "function_call" not in last_message.additional_kwargs:
    return "end"

  return "continue"

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : END
    }
)

Adding the edge which connects the action node to the agent node closing the loop or cycle.

In [ ]:
workflow.add_edge("action", "agent")

Compiling the workflow

In [ ]:
app = workflow.compile()

## Using the Graph and Testing

Testing the graph with a question that will trigger the wikidata tool.

In [ ]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who is the Spainish artist known for surrealist art in the 1920s?")]}

await app.ainvoke(inputs)

{'messages': [HumanMessage(content='Who is the Spainish artist known for surrealist art in the 1920s?'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Spanish artist known for surrealist art in the 1920s"}', 'name': 'duckduckgo_search'}}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 168, 'total_tokens': 197}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'function_call', 'logprobs': None}),
  FunctionMessage(content="Salvador Dalí (born May 11, 1904, Figueras, Spain—died January 23, 1989, Figueras) was a Spanish Surrealist painter and printmaker, influential for his explorations of subconscious imagery. Salvador Dalí and Man Ray. Salvador Dalí (left) and Man Ray, 1934. As an art student in Madrid and Barcelona, Dalí assimilated a vast number of ... Salvador Domingo Felipe Jacinto Dalí i Domènech, Marquess of Dalí of Púbol [a] gcYC (11 May 1904 - 23 January 1989), known as Sal

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "function_call" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "function_call" `additional_kwarg` and passed the state object to END where we see it output in the cell above!


Testing with the second tool Duck Duck Go Search

In [ ]:
inputs = {"messages" : [HumanMessage(content="What is QLoRA in Machine Learning? Are their any papers that could help me understand?")]}

await app.ainvoke(inputs)

{'messages': [HumanMessage(content='What is QLoRA in Machine Learning? Are their any papers that could help me understand?'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"QLoRA in Machine Learning"}', 'name': 'duckduckgo_search'}}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 169, 'total_tokens': 191}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'function_call', 'logprobs': None}),
  FunctionMessage(content='Large Language Models (LLMs) are currently a hot topic in the field of machine learning. Imagine you\'re an ML Engineer and your company has access to GPUs and open-source LLMs like LLAMA/Falcon. ... LoRA, Machine Learning, QLoRA, transformers. Categories: data-science, machine-learning. Updated: July 26, 2023. Share on Twitter Facebook ... Our results show that QLoRA finetuning on a small high-quality dataset leads to state-of-the-art results, even when using smaller 

This notebook is adapted from the notebook developed by AI Makerspace, which was originally using ArXiv and Duck Duck Go Search tools.